In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

 Import Libraries

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Load Dataset

In [4]:
dataset = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
df = dataset.copy()
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Preparing Features and Labels

In [5]:
# Prepare features (X) and labels (y)
X = df.drop('Class', axis=1)
y = df['Class']

Handling Class Imbalance using SMOTE

In [6]:
# Apply SMOTE to handle class imbalance
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)
X_resampled = X_resampled.values

Scaling and PCA Transformation

In [7]:
# Scale the numerical features
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[['Time', 'Amount'] + [f'V{i}' for i in range(1, 29)]] = scaler.fit_transform(df[['Time', 'Amount'] + [f'V{i}' for i in range(1, 29)]])

# Apply PCA to reduce dimensionality
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_resampled)

Setting Up k-Fold Cross-Validation

In [8]:
# Set up k-fold cross-validation
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

Initializing Lists to Store Results

In [9]:
# Initialize lists to store results
ann_accuracies = []

Training and Evaluating Models Using k-Fold Cross-Validation

In [10]:
# Perform k-fold cross-validation
for train_index, test_index in skf.split(X_resampled, y_resampled):
    X_train_fold, X_test_fold = X_resampled[train_index], X_resampled[test_index]
    y_train_fold, y_test_fold = y_resampled[train_index], y_resampled[test_index]

    # ANN model training
    ann_model = Sequential()
    ann_model.add(Dense(16, input_dim=X_train_fold.shape[1], activation='relu'))
    ann_model.add(Dense(8, activation='relu'))
    ann_model.add(Dense(1, activation='sigmoid'))

    ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    ann_model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)

    # Evaluate ANN model
    y_pred_ann_fold = (ann_model.predict(X_test_fold) > 0.5).astype("int32")
    ann_accuracies.append(ann_model.evaluate(X_test_fold, y_test_fold, verbose=0)[1])
    
    print("ANN Model Fold Result:")
    print(classification_report(y_test_fold, y_pred_ann_fold))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
ANN Model Fold Result:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     56863
           1       0.99      0.97      0.98     56863

    accuracy                           0.98    113726
   macro avg       0.98      0.98      0.98    113726
weighted avg       0.98      0.98      0.98    113726



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
ANN Model Fold Result:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     56863
           1       0.99      0.96      0.97     56863

    accuracy                           0.98    113726
   macro avg       0.98      0.98      0.98    113726
weighted avg       0.98      0.98      0.98    113726



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 974us/step
ANN Model Fold Result:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     56863
           1       0.98      0.97      0.98     56863

    accuracy                           0.98    113726
   macro avg       0.98      0.98      0.98    113726
weighted avg       0.98      0.98      0.98    113726



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
ANN Model Fold Result:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     56863
           1       1.00      0.90      0.95     56863

    accuracy                           0.95    113726
   macro avg       0.95      0.95      0.95    113726
weighted avg       0.95      0.95      0.95    113726



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
ANN Model Fold Result:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97     56863
           1       0.99      0.94      0.96     56863

    accuracy                           0.97    113726
   macro avg       0.97      0.97      0.97    113726
weighted avg       0.97      0.97      0.97    113726



Calculating and Printing Average Accuracy

In [11]:
# Print cross-validation results
print(f"ANN Model Average Accuracy: {np.mean(ann_accuracies):.4f}")

ANN Model Average Accuracy: 0.9693
